## To lauch the notebook select "Run All" from the "Cell" menu above.

# Eye Diagrams
<p style="font-size:18px">
This notebook demonstrates how amplitude noise, bandwidth limitations, and timing jitter lead to errors in determining the value of a transmitted bit, and how an eye diagram provides a visual illustration of the contributions to errors.

## Amplitude Noise, Q-Factor, and Bit Error Rate
<p style="font-size:18px">
One of the most important contributions to errors in the detection of optical signals is thermal noise in the optical receiver. Thermal noise arises from fluctuations in the current through a resitive load. It has a Gaussian distribution that is characterized by the standard deviation for the noise current, given by the expression
<p style="font-size:18px">
$${\sigma _T} = \sqrt {\frac{{4{k_B}T}}{{{R_L}}}\Delta f}, $$
where $k_B$ is Boltzman's constant, T is the temperature, $R_L$ is the resistance of the load, and $\Delta f$ is the bandwidth of the receiver. Current variation leads to a variation in the voltage across the resisttor and and to a variation in the amplitude of the measured signal.
<p style="font-size:18px">
Thermal noise constibutes equally to variation in the amplitude for transmitted "ones" and "zeros". More generally there are additional noise sources that contibute in different amounts to ones and zeros (shot noise is an example), and that are characterized by two different standard deviations ${\sigma _1}$ and ${\sigma _0}$.
<p style="font-size:18px">
A quantity called the Q-Factor is the generalization for signal-to-noise-ratio that is used when the amount of noise depends on signal level. The Q-Factor is given by the expression
<p style="font-size:18px">
$$Q \equiv \frac{{{I_1} - {I_0}}}{{{\sigma _0} + {\sigma _1}}},$$
<p style="font-size:18px">
where $I_1$ and $I_0$ are the average current generated by a transmitted one and zero, respectively. The bit error rate (BER, the fraction of bits incorrectly detected) is related to Q-Factor  by the expression
<p style="font-size:18px">
$$BER = \frac{1}{2}erfc\left( {\frac{Q}{{\sqrt 2 }}} \right),$$
where erfc is the complementary error function.

### Exploration
<p style="font-size:18px">
Explore how signal ampliture contibutes to the bit-error rate with the interactive graphs below. The graph on the left shows a signal that represents the bit sequence 0,1,0, as indicated in the figure. The graph on the right shows the distribution of the measured values for this sequence. A transmitted bit is determined to be a one if a measurment is above the voltage threshold, and a zero if a measurement is below the threshold. Signals are measured at the middle of a bit period. As is particulary apparent from the graph on the rigth, there is always a non-zero probability that a signal will be on the wrong side of the threshold level during a measurement, resulting in an incorrect translation of a signal into a bit.
<p style="font-size:18px">
Suggestion: Adjust the average received power to achieve a bit error rate of 1 x $10^{-9}$. Can you see the signal drop below the threshold value? Why or why not?
<p style="font-size:18px">
Suggestion: Adjust the singal slider so that the signal level foccasionally falls below the threshold level. What is the bit error rate in this case?

In [1]:
%run load_modules.ipynb

Loading BokehJS ...

In [2]:
%run err_dist_chart.ipynb

interactive(children=(FloatSlider(value=10.0, description='Pave (μW)', max=18.0, step=1.0), Output()), _dom_cl…

## Finite Bandwidth and Signal Jitter
<p style="font-size:18px">
The bandwidth of optical receivers is typically limited to the bit rate. For example, the bandwidth for an optical receiver for detecting a bit stream at 2.5 gigabits per second will be on the order of 2.5 GHz. A finite bandwidth has the effect of reducing thermal noise and error rates, as can be seen from the equation for $\sigma_T$ given above. On the other hand, finite bandwidth "rounds" signals and reduces the "flat" regions in the middle of bit periods where measurments take place. When combined with signal jitter (a random error in the timing of the transitions between bit periods), this can lead to increased error rates.
    
### Exploration
<p style="font-size:18px">
Explore how combinations of bandwidth and timing jitter distort signals. Just below, the curve on the left shows the frequency components for a square wave signal and added thermal noise contribution. The graph also shows the filtering reponse of an optical receiver with a 3dB bandwdith set by a slider. The graph on the right shows the filtered/bandwidth-limited signal.

<p style="font-size:18px">
Suggesion: Find a combination of bandwidth and jitter for which you can see the signal fall below (or almost fall below) the voltage threshold at the center of the bit period for the "1" (i.e. at 600 picoseconds).

In [7]:
%run bandwidth_jitter_chart.ipynb

interactive(children=(FloatSlider(value=10.0, description='3dB BW (GHz)', max=19.0, min=0.5, step=0.5, style=S…

## Building an Eye Diagram
<p style="font-size:18px">
A measurment of bit error rate provides a single, quantitative value that provides a valuable, overall assessment of the peformance of an optical link. An eye diagram is a visual tool that can be used to quickly produce a qualtitative assessment of the various contributions to the bit error rate, including themal noise, finite bandwidth, and timing jitter. An eye diagram is created by overlapping multiple signal traces, each carrying a separate random bit pattern. The traces are typically two bit periods long, with half a bit period followed by a full bit period, followed by half a bit period.

### Exploration
<p style="font-size:18px">
The graph just below shows how an eye diagram is built up by drawing multiple segments of bit stream. Click on the "Trace" button to draw a signal curve on the graph. The random bit pattern will be one of eight - 000. 001, 010, 011, 100, 101, 110, or 111. Up to 21 curves can be drawn on the graph. The "Reset" button clears the graph.

<p style="font-size:18px">
Suggestion: Find the minimum number of signal traces needed to create an eye diagram that shows both the vertical and horizontal undertainty in the signal values.

In [ ]:
%run eye_diagram_chart.ipynb